# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,8.22,79,6,3.74,ZA,1715484974
1,1,ushuaia,-54.8000,-68.3000,3.81,56,40,7.72,AR,1715484974
2,2,anadyr,64.7500,177.4833,-0.72,74,100,2.00,RU,1715484975
3,3,bilibino,68.0546,166.4372,2.99,76,75,1.19,RU,1715484975
4,4,enewetak,11.3474,162.3373,27.70,81,94,8.14,MH,1715484975


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.6
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & 
                                     (city_data_df["Max Temp"] < 27) & 
                                     (city_data_df["Wind Speed"] < 4.5) & 
                                     (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna(how='any')

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
62,62,morondava,-20.2833,44.2833,21.99,65,0,3.00,MG,1715484931
91,91,pierre,44.3683,-100.3510,21.51,56,0,2.57,US,1715484993
120,120,marsa alam,25.0757,34.8918,26.28,69,0,4.26,EG,1715484998
139,139,arkansas city,37.0620,-97.0384,21.79,72,0,1.54,US,1715485001
154,154,orange city,28.9489,-81.2987,23.97,85,0,0.00,US,1715485004
162,162,cockburn town,21.4612,-71.1419,26.26,84,0,2.44,TC,1715485005
195,195,tamanrasset,22.7850,5.5228,21.95,15,0,2.57,DZ,1715485012
216,216,revelstoke,50.9832,-118.2023,21.01,32,0,3.09,CA,1715485016
221,221,ghat,24.9647,10.1728,21.43,25,0,1.58,LY,1715484792
241,241,brandon,27.9378,-82.2859,26.03,78,0,0.45,US,1715485021


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
62,morondava,MG,-20.2833,44.2833,65,
91,pierre,US,44.3683,-100.3510,56,
120,marsa alam,EG,25.0757,34.8918,69,
139,arkansas city,US,37.0620,-97.0384,72,
154,orange city,US,28.9489,-81.2987,85,
162,cockburn town,TC,21.4612,-71.1419,84,
195,tamanrasset,DZ,22.7850,5.5228,15,
216,revelstoke,CA,50.9832,-118.2023,32,
221,ghat,LY,24.9647,10.1728,25,
241,brandon,US,27.9378,-82.2859,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
morondava - nearest hotel: Hotel Sharon Menabe
pierre - nearest hotel: Quality Inn
marsa alam - nearest hotel: Blue House Hotel
arkansas city - nearest hotel: Best Western Atrium Gardens
orange city - nearest hotel: 1876 Heritage Inn
cockburn town - nearest hotel: The Salt Raker Inn
tamanrasset - nearest hotel: فندق الأمان
revelstoke - nearest hotel: Grizz Hotel
ghat - nearest hotel: فندق تاسيلي
brandon - nearest hotel: Brandon Motor Lodge
truth or consequences - nearest hotel: Travelodge
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
marianna - nearest hotel: Best Value Inn Motel Sandusky
namibe - nearest hotel: Nelsal Pensao
gurupi - nearest hotel: Via Norte
abnub - nearest hotel: University Guest House
nova xavantina - nearest hotel: Hotel Morada do Sol
ixtapa - nearest hotel: Ma. Cristina
seminole - nearest hotel: Best Western Plus Sanford
forest grove - nearest hotel: Best Western
miracema do tocantins - nearest hotel: No hotel found
yinzhu

,City,Country,Lat,Lng,Humidity,Hotel Name
62,morondava,MG,-20.2833,44.2833,65,Hotel Sharon Menabe
91,pierre,US,44.3683,-100.3510,56,Quality Inn
120,marsa alam,EG,25.0757,34.8918,69,Blue House Hotel
139,arkansas city,US,37.0620,-97.0384,72,Best Western Atrium Gardens
154,orange city,US,28.9489,-81.2987,85,1876 Heritage Inn
162,cockburn town,TC,21.4612,-71.1419,84,The Salt Raker Inn
195,tamanrasset,DZ,22.7850,5.5228,15,فندق الأمان
216,revelstoke,CA,50.9832,-118.2023,32,Grizz Hotel
221,ghat,LY,24.9647,10.1728,25,فندق تاسيلي
241,brandon,US,27.9378,-82.2859,78,Brandon Motor Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.6,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)